
----
##Goal is to implement a multi-class Neural network Classifier. 




In [ ]:
%pylab inline
import scipy.stats
from collections import defaultdict  # default dictionary 

In [ ]:
#Cutomize the Matplotlib for beautiful plots...
#comment it if it does not work for you.
#import dmStyle
#dmStyle.customize_mpl()


In [ ]:
import pandas as pd
import tools as t # set of tools for plotting, data splitting, etc..

In [ ]:
import time 
def GradientDescent(X,Y,cost_function,derivative_cost_function,nparams,maxniter=20000, alpha=0.01, plotcf=True):
    '''
        Finds the minimum of given cost function using gradient descent.
        
        Input:
        ------
            X: can be either a single d X m-dimensional vector 
                or d X m dimensional matrix of inputs            
            
            Y: Must be k X m - dimensional label vector
            cost_function: a function to be minimized, must return a scalar value
            derivative_cost_function: derivative of cost function w.r.t. paramaters, 
                                       must return partial derivatives w.r.t all d parameters
            aparams : number of parameters (length of theta vector to learn...)                                       
            maxniter: maximum number of iterations to run
            alpha: learning rate
            plotcf: If true, plots cost function after every 20 iterations
                    
        Returns:
        ------
            thetas: a nclass*d X 1-dimensional vector of cost function parameters 
                    where minimum point occurs (or location of minimum).
    '''
    import pdb
    # Remember you must plot the cost function after set of iterations to
    # check whether your gradient descent code is working fine or not...
    eps=0.00001
    thetas=rand(nparams,1)
    # lets map the thetas to the range [-1,1]
    stime=time.time() # Get the Starting time...
    
    thetas= thetas*2-1  # output = input * range_output  + min_output

    theta_old=thetas
    theta_new=thetas+1

#     pdb.set_trace()
    print thetas.shape,theta_old.shape,theta_new.shape
    cf=[cost_function(X,Y,theta_old)]
#     pai=int(maxniter / 200) # plot in total 200 number of points
    #print cf
    numiter=0
    tnorms=[]
    df=ones(thetas.shape)
    while numiter < maxniter and np.linalg.norm(df) > eps:
        theta_old=theta_new
        theta_new=theta_old-alpha*np.array(derivative_cost_function(X,Y,np.array(theta_old)))
        numiter+=1
        print 'Value of Cost Function at Minimum Points {}, is {}'.format(theta_new,cost_function(X,Y,theta_new))
    
    # print cf

    #print theta_new
    print 'Total time taken ={:.4} seconds '.format(time.time()-stime) # difference between current and stored time
    return theta_new


In [ ]:
import pdb
def one_of_k(ilabels):
    """
        Computes one of k representation for the given labels
        
        Input:
            ilabels: class labels (m x 1)
        
        output:
             
            olabels: one of k representation of 
    """
    
    classes=np.unique(ilabels)
    
    nclasses=len(classes)
    
    olabels=np.zeros((ilabels.shape[0],nclasses))
    
    for i, l in enumerate(ilabels):        
        olabels[i,classes==l]=1
        
    return olabels

In [ ]:
# A placeholder class 
# TODO: You have to implement the following class

### import pdb
## Your code goes here...
# You might need to define auxliary classes for composition.. ?

class NeuralNetwork:
    ''' Implements the multiclass Neural Network For Classification... 

        It is  a simple implementation of 3 Layer NeuralNetwork, for learning purpose,
        you can extend it to multi-layer with some simple modifications....


    '''

    def __init__(self,ifeatdim,nnhlayer,classes,alpha=0.001,maxniter=20000,scalefeatures=False):        
        self.thetas=[] # learned set of parameters
        self.scalefeatures=scalefeatures
        self.classes=np.unique(classes)

        self.nclasses=len(self.classes) # dimensions of output
        self.ifeatdim=ifeatdim # dimension of input features,
        self.nnhlayer=nnhlayer # number of neurons in hidden layer
        
        self.maxniter=maxniter
        self.alpha=alpha

        self.sl1theta= (nnhlayer, ifeatdim) # l1theta's shape, remember 1 is already added for bias terms
        self.sl2theta= (self.nclasses, nnhlayer+1) # l2thetas's shape, +1 for bias terms

        # thetas's for neurons + bias terms..
        
        self.nthetas= (self.nclasses * nnhlayer+ nnhlayer* self.ifeatdim) + (self.nclasses)
        
        assert(self.nthetas == np.product(self.sl1theta[:])+np.product(self.sl2theta[:])) # for debugging 

        pass
    
    def getnparam(self):
        """
        
            returns number of parameters in the network...
        """
        
        return self.nthetas
    def unroll_thetas(self,thetas):
        """
            unroll (unpacks) a long vector of thetas to respective layer thetas...
            we will be using a long vector of thetas during Gradient Descent but
            during forward_pass and back_ward pass we will unpack this theta into 
            thetas of respective layers
            
            Input:
            --------
                a long vector of self.nthetas dimensions
            Output:
                a tuple with first element contains the thetas of first layer
                and second element contains the thetas of second layer.
        """

        l1theta= thetas[:np.product(self.sl1theta)].reshape(self.sl1theta)

        l2theta= thetas[np.product(self.sl1theta):].reshape(self.sl2theta)

        return l1theta, l2theta

    def roll_thetas(self,l1theta,l2theta):

        """
            Rolls (packs) the layer-wise thetas to a single long vector of thetas...

            Input:
            --------
                l1theta: thetas of layer-1
                l2theta: thetas of layer-2

            Output:
                a long vector of thetas containing both layers thetas
        """

        return np.concatenate( ( l1theta.ravel(),l2theta.ravel() )  )

    def forward_pass(self, X, thetas):
        """
            Compute the forward_pass across the layers...
            Input:
            X: training matrix (d X m)
            thetas: a long vector of thetas
        """
        activations=[]
        a2=[]
        a3=[]
        weightedSums=[]
        l1theta,l2theta=self.unroll_thetas(thetas) # unroll the thetas...
        # layer 1 computations...
        a2=l1theta.dot(X)
        # append 1's for the bias terms...
        a2=np.vstack((a2,np.ones((1,a2.shape[1]))))
        
        #layer 2 computations...
        a3=l2theta.dot(self.sigmoid(a2))
        

        return self.sigmoid(a2),self.sigmoid(a3)
    def sigmoid(self,z):
        """
            Compute the sigmoid function 
            Input:
                z can be a scalar or a matrix
            Returns:
                sigmoid of the input variable z
        """
        #clip the z

        z = maximum(minimum(16,z),-16) 
        return 1/(1+np.exp(-z))        
    
    
    def hypothesis(self, X,thetas):
        '''
            Computes the hypothesis for  given input examples (X) and parameters (thetas).

            Input:
                X: can be either a single d X n-dimensional vector or d X n dimensional matrix
                thetas: Must be a d-dimensional vector
            Return:
                The computed hypothesis
        '''
    
        return (self.forward_pass(X,thetas))[1]
    
    def cost_function(self, X,Y, thetas):
        '''
            Computes the Cost function for given input data (X) and labels (Y).

            Input:
                X: can be either a single d X n-dimensional vector or d X n dimensional matrix of inputs
                theata: must  dk X 1-dimensional vector for representing vectors of k classes
                Y: Must be k X n-dimensional label vector
                
            Return:
                Returns the cost of hypothesis with input parameters 
        '''
        m=X.shape[1]
        
        h= self.hypothesis(X,thetas)
        cost=np.sum((-1)*Y*np.log10(h)-(1-Y)*np.log10(1-h))/float(m)
        
        print "cost : ",cost
        return cost
    def derivative_cost_function(self,X,Y,thetas):
        '''
            Computes the derivates of Cost function w.r.t input parameters (thetas)  
            for given input and labels.

            Input:
            ------
                X: can be either a single d X n-dimensional vector or d X n dimensional matrix of inputs
                theata: must  dk X 1-dimensional vector for representing vectors of k classes
                Y: Must be k X n-dimensional label vector
            Returns:
            ------
                partial_thetas: a dk X 1-dimensional vector of partial derivatives of cost function w.r.t parameters..
        '''
#       pdb.set_trace()
        #forward pass
        a2, a3=self.forward_pass(X,thetas)
        
        #now back-propogate 

        # unroll thetas
        l1theta,l2theta=self.unroll_thetas(thetas) # unroll the thetas...
        
        nexamples=float(X.shape[1])
        
        # compute delta3, l2theta
        delta3=(a3-Y)
        delta3/=nexamples
        
        derivativeOfDelta3=(delta3.dot(a2.T))
        #print derivativeOfDelta3.shape
        # compute delta2, l1 theta
        delta2=((l2theta.T).dot(delta3))
        delta2*=(a2*(1-a2))
        delta2=delta2[:-1]
        derivativeOfDelta2= delta2.dot(X.T)
      
        #remember to exclude last element of delta2, representing the deltas of bias terms...
        # i.e. delta2=delta2[:-1]
        l1theta=derivativeOfDelta2
        l2theta=derivativeOfDelta3
        
        print "l1theta : ",l1theta.shape
        print "l2theta : ",l2theta.shape

        # roll thetas into a big vector
        thetas=(self.roll_thetas(l1theta,l2theta)).reshape(thetas.shape) # return the same shape as you received
        
        return thetas
        #return partialderivatives

    def train(self, X, Y):
        ''' Train LogisticRegression classifier using the given 
            X [d x m] data matrix and Y labels matrix
            
            Input:
            ------
            X: [d x m] a data matrix of m d-dimensional examples.
            Y: [k x m] a label vector.
            
            Returns:
            -----------
            Nothing
            '''
        
        
        nfeatures,nexamples=X.shape
        ## now go and train a model for each class...
        if self.scalefeatures:
            X=self.scale_features(X)
        nY=one_of_k(Y.T) # get a n x k
        nparams=self.getnparam()
        print 'Calling Gradient Descent with following number of parameter={}'.format(nparams)
        self.thetas=GradientDescent(X,nY.T,self.cost_function,self.derivative_cost_function,self.getnparam(), self.maxniter,self.alpha)

    def predict(self, X):
        
        """
        Test the trained NeuralNetwork classifier result on the given examples X
        
                   
            Input:
            ------
            X: [d x m] a matrix of m  d-dimensional test examples.
           
            Returns:
            -----------
            pclass: the predicted class for the given set of examples, i.e. to which it belongs
        """
        
        num_test = X.shape[1]
        
        if self.sl1theta[1]-X.shape[0] ==1:
            # append 1 at the end of each example for the bias term
            X=np.vstack((X,np.ones((1,X.shape[1]))))
        
        if self.scalefeatures:
            X=(X-self.xmin)/(self.xmax-self.xmin)
        
       
        Ypred=[]

        h=self.hypothesis(X.dot(theta))
        pos=np.argmax(h,axis=-1)
        Ypred=self.classes[pos]
        
        return np.array(Ypred).reshape((num_test,1))
    
    
    def scale_features(self,X):
        """
            Normalize each feature to lie in the range [0 ,1]

            Input:
            ------

                X= d x M dimensional data matrix

            Returns:
            --------

                normalized X
        """
        self.xmin= np.min(X,axis=1)
        self.xmax= np.max(X,axis=1)

        return (X-self.xmin)/(self.xmax-self.xmin)

#Non-Linear Binary Class Problem...

In [ ]:
# Create some dummy data for testing

np.random.seed(seed=99)

# make some data up
cp=5
nclasses=2
mean1 = [-cp,-cp]
mean2 = [cp,cp]
mean3 = [cp,-cp]
mean4 = [-cp,cp]
cov = [[3.0,0.0],[0.0,3.0]] 

#create some points
nexamples=1000
x1 = np.random.multivariate_normal(mean1,cov,nexamples/4)
x2 = np.random.multivariate_normal(mean2,cov,nexamples/4)
x3 = np.random.multivariate_normal(mean3,cov,nexamples/4)
x4 = np.random.multivariate_normal(mean4,cov,nexamples/4)

X=np.vstack((x1,x2,x3,x4))
Y=np.vstack((1*np.ones((nexamples/4,1)),1*np.ones((nexamples/4,1)),2*np.ones((nexamples/4,1)),2*np.ones((nexamples/4,1))))

plt.scatter(x1[:,0],x1[:,1], c='r', s=100)
plt.scatter(x2[:,0],x2[:,1], c='r', s=100)            
plt.scatter(x3[:,0],x3[:,1], c='b', s=100)
plt.scatter(x4[:,0],x4[:,1], c='b', s=100)            



plt.title("Linear Classification")
plt.xlabel("feature $x_1$")
plt.ylabel("feature $x_2$")

fig_ml_in_10 = plt.gcf()
plt.savefig('multi-linear-class-nn.svg',format='svg')

In [ ]:
print X.shape,Y.shape

In [ ]:
#Lets append a vector of dummy 1's at the end of X to simplify the calculations...
X=np.hstack((X,np.ones((X.shape[0],1))))

In [ ]:
print X.shape,Y.shape

In [ ]:
ook=one_of_k(Y)

In [ ]:
ook

In [ ]:
#
def test_derivative(X,Y,cost_function,derivative_cost_function,nparams):
    """
        Function test the implementation of the derivative function...
        X: Input test examples (m x d)
        Y: True labels (m x 1)
        cost_function: function used to compute cost function...
        derivative_cost_function: function used to compute derivative of the cost function...
        nparams: number of parameters in the network...
    """
    thetas=np.random.rand(nparams,1)
#     import pdb
#     pdb.set_trace()
    ad=derivative_cost_function(X,Y,thetas)
    print ad
    eps=0.0001
    cd=[]

    for i in range(len(thetas)):
        ttpe=thetas.copy()
        ttpe[i]=ttpe[i]+eps # add an epsilon for the current theta
        ttme=thetas.copy()
        ttme[i]=ttme[i]-eps # add an epsilon for the current theta
#         print 'diff in direction', ttpe-ttme    
        cd.append((cost_function(X,Y,ttpe)-cost_function(X,Y,ttme))/(2*eps))

    print 'Computational derivatvie =',cd
    print 'Analytical derivative =', ad.shape,ad.T
    print 'Their difference=',np.reshape(cd,ad.shape)-ad

In [ ]:
%pdb off
nnclass=NeuralNetwork(3,2,[1, 2],alpha=0.01)

In [ ]:
print "Shape of Parameters in L1 of ={}".format(nnclass.sl1theta)
print "Shape of Parameters in L2 of ={}".format(nnclass.sl2theta)
print "Number of Parameters in Network ={}".format(nnclass.getnparam())

In [ ]:
print ook.shape


In [ ]:
# %reset
# we will be sending transpose of the elements to make sure the data is presented in d X m format
%pdb off
test_derivative(X.T,ook.T,nnclass.cost_function,nnclass.derivative_cost_function,nnclass.getnparam())

In [ ]:
%pdb
#once again sending the transpose of the training and testing matrix
nnclass.train(X.T,Y.T) # c

In [ ]:
%pdb off
npts=10000
model=nnclass
ax=plt.gca()
x0spr = max(X[:,0])-min(X[:,0])
x1spr = max(X[:,1])-min(X[:,1])

tx=np.random.rand(npts,2)
tx[:,0] = tx[:,0]*x0spr + min(X[:,0])
tx[:,1] = tx[:,1]*x1spr + min(X[:,1])

print tx.shape

cs= model.predict(tx.T); # send the transpose of X during evaluation as well...
print  np.unique(cs)
ax.scatter(tx[:,0],tx[:,1],c=cs.ravel(), alpha=.25)

ax.hold(True)
ax.scatter(X[:,0],X[:,1],
              c=list(map(lambda x:'blue' if x==1 else 'lime' if x==2 else 'r',Y)), 
              linewidth=0,s=25,alpha=1)
ax.set_xlim([min(X[:,0]), max(X[:,0])])
ax.set_ylim([min(X[:,1]), max(X[:,1])])

#Three class problem

In [ ]:
# Create some dummy data for training and testing

np.random.seed(seed=99)

# make some data up using Gaussian distribution...
mean1 = [-3,-3]
mean2 = [2,2]
mean3 = [-4,5]
cov = [[1.0,0.0],[0.0,1.0]] 

#create some points
nexamples=600
x1 = np.random.multivariate_normal(mean1,cov,nexamples/3)
x2 = np.random.multivariate_normal(mean2,cov,nexamples/3)
x3 = np.random.multivariate_normal(mean3,cov,nexamples/3)

X=np.vstack((x1,x2,x3))
Y=np.vstack((1*np.ones((nexamples/3,1)),2*np.ones((nexamples/3,1)),3*np.ones((nexamples/3,1))))

plt.scatter(x1[:,0],x1[:,1], c='r', s=100)
plt.scatter(x2[:,0],x2[:,1], c='b', s=100)            
plt.scatter(x3[:,0],x3[:,1], c='g', s=100)



plt.title("Linear Classification")
plt.xlabel("feature $x_1$")
plt.ylabel("feature $x_2$")

fig_ml_in_10 = plt.gcf()
plt.savefig('multi-linear-class-nn.svg',format='svg')

In [ ]:
print X.shape,Y.shape

In [ ]:
#Lets append a vector of dummy 1's at the end of X to simplify the calculations...
X=np.hstack((X,np.ones((X.shape[0],1))))

In [ ]:
ook=one_of_k(Y)

In [ ]:
%pdb off
nnclass=NeuralNetwork(3,2,np.unique(Y),alpha=0.01)

In [ ]:
# %reset
%pdb off
test_derivative(X.T,ook.T,nnclass.cost_function,nnclass.derivative_cost_function,nnclass.getnparam())

In [ ]:
nnclass.train(X.T,Y.T) # c

## Let Plots the decision boundaries of the classes, using simple scatter plots... 

In [ ]:
%pdb off
npts=10000
model=nnclass
ax=plt.gca()
x0spr = max(X[:,0])-min(X[:,0])
x1spr = max(X[:,1])-min(X[:,1])

tx=np.random.rand(npts,2)
tx[:,0] = tx[:,0]*x0spr + min(X[:,0])
tx[:,1] = tx[:,1]*x1spr + min(X[:,1])

print tx.shape
#pdb.set_trace()
cs= model.predict(tx.T);
print cs
print  np.unique(cs)
ax.scatter(tx[:,0],tx[:,1],c=cs.ravel(), alpha=.35)

ax.hold(True)
ax.scatter(X[:,0],X[:,1],
              c=list(map(lambda x:'blue' if x==1 else 'lime' if x==2 else 'r',Y)), 
              linewidth=0,s=25,alpha=0.75)
ax.set_xlim([min(X[:,0]), max(X[:,0])])
ax.set_ylim([min(X[:,1]), max(X[:,1])])

#Load Iris dataset and test the classifier on them...

In [ ]:
#load the data set
data=pd.read_csv('./data/iris.data')
data.columns=['SepalLength','SepalWidth','PetalLength','PetalWidth','Class']
print data.describe()

In [ ]:
# Get your data in matrix
X=np.asarray(data[['SepalLength','SepalWidth','PetalLength','PetalWidth']].dropna())
Y=np.asarray(data['Class'].dropna())
print " Data Set Dimensions=", X.shape, " True Class labels dimensions", Y.shape

In [ ]:
#Lets append a vector of dummy 1's at the start of X to simplify the calculations...
X=np.hstack((X,np.ones((X.shape[0],1))))

In [ ]:
feat=[2,3,4]
Y=Y.reshape((len(Y),1))

In [ ]:
#%pdb
nnclass=NeuralNetwork(3,3,np.unique(Y),alpha=0.02)
nnclass.train(X[:,feat].T,Y.T)

In [ ]:
def print_confusion_matrix(plabels,tlabels):
    """
        functions print the confusion matrix for the different classes
        to find the error...
        
        Input:
        -----------
        plabels: predicted labels for the classes...
        tlabels: true labels for the classes
        
        code from: http://stackoverflow.com/questions/2148543/how-to-write-a-confusion-matrix-in-python
    """
    import pandas as pd
    plabels = pd.Series(plabels)
    tlabels = pd.Series(tlabels)
    
    # draw a cross tabulation...
    df_confusion = pd.crosstab(tlabels,plabels, rownames=['Actual'], colnames=['Predicted'], margins=True)
    
    #print df_confusion
    return df_confusion

In [ ]:
print_confusion_matrix((nnclass.predict(X[:,feat].T)).ravel(),Y.ravel())

###Lets plot the decision boundaries

In [ ]:
npts=10000
model=nnclass
ax=plt.gca()
x0spr = max(X[:,feat[0]])-min(X[:,feat[0]])
x1spr = max(X[:,feat[1]])-min(X[:,feat[1]])

tx=np.random.rand(npts,2)
tx[:,0] = tx[:,0]*x0spr + min(X[:,feat[0]])
tx[:,1] = tx[:,1]*x1spr + min(X[:,feat[1]])

print tx.shape
#pdb.set_trace()
cs= model.predict(tx.T);
# print cs
print  np.unique(cs)
mc=list(map(lambda x:1 if x=='Iris-setosa' else 2 if x=='Iris-versicolor' else 3,cs))
ax.scatter(tx[:,0],tx[:,1],c=mc, alpha=.25)

ax.hold(True)
ax.scatter(X[:,feat[0]],X[:,feat[1]],
               c=list(map(lambda x:'blue' if x=='Iris-setosa' else 'lime' if x=='Iris-versicolor' else 'r',Y)), 
               linewidth=0,s=25,alpha=0.8)
ax.set_xlim([min(X[:,feat[0]]), max(X[:,feat[0]])])
ax.set_ylim([min(X[:,feat[1]]), max(X[:,feat[1]])])